In [1]:
# Import necessary libraries
import os
import cv2
import numpy as np
import tensorflow as tf
from glob import glob
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Conv2D, Activation, BatchNormalization, UpSampling2D, Input, Concatenate, MaxPool2D
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, CSVLogger, TensorBoard
from tensorflow.keras.metrics import Recall, Precision
from tqdm import tqdm

In [2]:
def load_data(scan_folder, mask_folder, split=0.1):
    images = sorted(glob(os.path.join(scan_folder, "*")))
    masks = sorted(glob(os.path.join(mask_folder, "*")))

    min_length = min(len(images), len(masks))
    images = images[:min_length]
    masks = masks[:min_length]

    total_size = len(images)
    valid_size = int(split * total_size)
    test_size = int(split * total_size)

    train_x, valid_x = train_test_split(images, test_size=valid_size, random_state=42)
    train_y, valid_y = train_test_split(masks, test_size=valid_size, random_state=42)

    train_x, test_x = train_test_split(train_x, test_size=test_size, random_state=42)
    train_y, test_y = train_test_split(train_y, test_size=test_size, random_state=42)

    return (train_x, train_y), (valid_x, valid_y), (test_x, test_y)
def read_image(PATH):
    PATH = PATH.decode()
    x = cv2.imread(PATH, cv2.IMREAD_COLOR)
    x = cv2.resize(x, (128, 128))
    x = x / 255.0
    return x

def read_mask(PATH):
    PATH = PATH.decode()
    x = cv2.imread(PATH, cv2.IMREAD_GRAYSCALE)
    x = cv2.resize(x, (128, 128))
    x = x / 255.0
    x = np.expand_dims(x, axis=-1)
    return x

def tf_parse(sx, sy):
    def _parse(x, y):
        x = read_image(x)
        y = read_mask(y)
        return x, y
    sx, sy = tf.numpy_function(_parse, [sx, sy], [tf.float64, tf.float64])
    sx.set_shape([128, 128, 3])
    sy.set_shape([128, 128, 1])
    return sx, sy

def tf_dataset(x, y, batch=8):
    dataset = tf.data.Dataset.from_tensor_slices((x, y))
    dataset = dataset.map(tf_parse)
    dataset = dataset.batch(batch)
    dataset = dataset.repeat()
    return dataset


In [3]:
# Model Definition
def conv_block(x, num_filters):
    x = Conv2D(num_filters, (3, 3), padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = Conv2D(num_filters, (3, 3), padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    return x

def build_model():
    size = 128
    num_filters = [64,128,256]
    inputs = Input(shape=(size, size, 3))
    skip_x = []
    x = inputs
    for f in num_filters:
        x = conv_block(x, f)
        skip_x.append(x)
        x = MaxPool2D((2, 2))(x)
    x = conv_block(x, num_filters[-1])
    num_filters.reverse()
    skip_x.reverse()
    for i, f, in enumerate(num_filters):
        x = UpSampling2D((2, 2))(x)
        xs = skip_x[i]
        x = Concatenate()([x, xs])
        x = conv_block(x, f)
    x = Conv2D(1, (1, 1), padding="same")(x)
    x = Activation("sigmoid")(x)
    return Model(inputs, x)


In [4]:
# Training
def iou(y_true, y_pred):
    def f(y_true, y_pred):
        intersection = (y_true * y_pred).sum()
        union = y_true.sum() + y_pred.sum() - intersection
        x = (intersection + 1e-15) / (union + 1e-15)
        x = x.astype(np.float32)
        return x
    return tf.numpy_function(f, [y_true, y_pred], tf.float32)

In [5]:
# Define paths and hyperparameters
scan_folder = '/content/drive/MyDrive/UNET-Data/Transversal'
mask_folder = '/content/drive/MyDrive/UNET-Data/Transversal-Mask'
BATCH = 8
lr = 1e-4
epochs = 50

# Load data
(train_x, train_y), (valid_x, valid_y), (test_x, test_y) = load_data(scan_folder, mask_folder)

# Prepare datasets
train_dataset = tf_dataset(train_x, train_y, batch=BATCH)
valid_dataset = tf_dataset(valid_x, valid_y, batch=BATCH)

# Build and compile model
model = build_model()
model.summary()
opt = tf.keras.optimizers.Adam(lr)
metrics = ["acc", Recall(), Precision(), iou]
model.compile(loss="binary_crossentropy", optimizer=opt, metrics=metrics)

# Set up callbacks
callbacks = [
    ModelCheckpoint("model.h5"),
    ReduceLROnPlateau(monitor="val_loss", factor=0.1, patience=3),
    CSVLogger("data.csv"),
    TensorBoard(),
    EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=False)
]

# Determine steps per epoch
train_steps = len(train_x) // BATCH
valid_steps = len(valid_x) // BATCH
if len(train_x) % BATCH != 0:
    train_steps += 1
if len(valid_x) % BATCH != 0:
    valid_steps += 1

# Train the model
model.fit(
    train_dataset,
    validation_data=valid_dataset,
    epochs=epochs,
    steps_per_epoch=train_steps,
    validation_steps=valid_steps,
    callbacks=callbacks
)

# Evaluation
def mask_parse(mask):
    mask = np.squeeze(mask)
    mask = [mask, mask, mask]
    mask = np.transpose(mask, (1, 2, 0))
    return mask

# Load the model for evaluation
model = tf.keras.models.load_model("model.h5", custom_objects={'iou': iou})

# Prepare test dataset
test_dataset = tf_dataset(test_x, test_y, batch=BATCH)
test_steps = len(test_x) // BATCH
if len(test_x) % BATCH != 0:
    test_steps += 1

# Evaluate the model
model.evaluate(test_dataset, steps=test_steps)



Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 128, 128, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 128, 128, 64)         1792      ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 128, 128, 64)         256       ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (None, 128, 128, 64)         0         ['batch_normalization[0][0

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


53/53 [==============================] - 178s 3s/step - loss: 0.2654 - acc: 0.9687 - recall: 0.1956 - precision: 0.0911 - iou: 0.0186 - val_loss: 0.4261 - val_acc: 0.9892 - val_recall: 0.0000e+00 - val_precision: 0.0000e+00 - val_iou: 0.0109 - lr: 1.0000e-04
Epoch 2/50
53/53 [==============================] - 8s 158ms/step - loss: 0.1236 - acc: 0.9913 - recall: 0.3694 - precision: 0.7622 - iou: 0.0407 - val_loss: 0.2855 - val_acc: 0.9892 - val_recall: 0.0000e+00 - val_precision: 0.0000e+00 - val_iou: 0.0124 - lr: 1.0000e-04
Epoch 3/50
53/53 [==============================] - 9s 165ms/step - loss: 0.1022 - acc: 0.9928 - recall: 0.5660 - precision: 0.7728 - iou: 0.0603 - val_loss: 0.1967 - val_acc: 0.9892 - val_recall: 0.0000e+00 - val_precision: 0.0000e+00 - val_iou: 0.0121 - lr: 1.0000e-04
Epoch 4/50
53/53 [==============================] - 7s 141ms/step - loss: 0.0875 - acc: 0.9939 - recall: 0.6235 - precision: 0.8305 - iou: 0.0767 - val_loss: 0.1699 - val_acc: 0.9892 - val_recall: 0.

[0.008355269208550453,
 0.9974745512008667,
 0.8843879103660583,
 0.9695421457290649,
 0.6356183290481567]

In [6]:
def read_image(PATH):
    x = cv2.imread(PATH, cv2.IMREAD_COLOR)
    x = cv2.resize(x, (128, 128))
    x = x / 255.0
    return x

def read_mask(PATH):
    x = cv2.imread(PATH, cv2.IMREAD_GRAYSCALE)
    x = cv2.resize(x, (128, 128))
    x = x / 255.0
    x = np.expand_dims(x, axis=-1)
    return x

def mask_parse(mask):
    mask = np.squeeze(mask)
    mask = np.stack([mask] * 3, axis=-1)
    return mask

In [7]:



# Load the model for evaluation
model = tf.keras.models.load_model("model.h5", custom_objects={'iou': iou})

# Create a directory to save the results
if not os.path.exists("Results"):
    os.makedirs("Results")

# Make predictions and visualize results
def predict_and_save_results(model, test_x, test_y):
    for i, (x_path, y_path) in tqdm(enumerate(zip(test_x, test_y)), total=len(test_x)):
        x = read_image(x_path)
        y = read_mask(y_path)
        y_pred = model.predict(np.expand_dims(x, axis=0))[0]
        y_pred = y_pred > 0.5

        h, w, _ = x.shape
        white_line = np.ones((h, 10, 3)) * 255.0
        all_images = [x * 255.0, white_line, mask_parse(y) * 255.0, white_line, mask_parse(y_pred) * 255.0]
        image = np.concatenate(all_images, axis=1)
        cv2.imwrite(f"Results/{i}.png", image)

# Predict and save results
predict_and_save_results(model, test_x, test_y)


  0%|          | 0/53 [00:00<?, ?it/s]

1/1 [==============================] - 2s 2s/step


  2%|▏         | 1/53 [00:01<01:35,  1.84s/it]

1/1 [==============================] - 0s 27ms/step


  4%|▍         | 2/53 [00:01<00:42,  1.20it/s]

...

 92%|█████████▏| 49/53 [00:06<00:00, 11.38it/s]

1/1 [==============================] - 0s 18ms/step


 96%|█████████▌| 51/53 [00:06<00:00, 11.71it/s]

1/1 [==============================] - 0s 18ms/step


100%|██████████| 53/53 [00:06<00:00,  7.68it/s]


In [8]:

from skimage import color
from skimage.io import imread


In [13]:
class Evaluate(object):
    def __init__(self, pred_image, gt_image):
        self.pred_image = pred_image
        self.gt_image = gt_image

    def cm_terms(self):
        n11 = n12 = n21 = n22 = 0
        rows, cols,_ = self.gt_image.shape

        for i in range(rows):
            for j in range(cols):
                if self.gt_image[i, j] == 0 and self.pred_image[i, j] == 0:
                    n11 += 1
                if self.gt_image[i, j] == 0 and self.pred_image[i, j] >= 1:
                    n12 += 1
                if self.gt_image[i, j] >= 1 and self.pred_image[i, j] == 0:
                    n21 += 1
                if self.gt_image[i, j] >= 1 and self.pred_image[i, j] >= 1:
                    n22 += 1

        return n11, n12, n21, n22

    def pixel_accuracy(self):
        n11, n12, n21, n22 = self.cm_terms()
        t1 = n11 + n12
        t2 = n21 + n22

        if (t1 + t2) == 0:
            pa = 0
        else:
            pa = float(n11 + n22) / float(t1 + t2)
        return pa * 100

    def mean_accuracy(self):
        n11, n12, n21, n22 = self.cm_terms()
        t1 = n11 + n12
        t2 = n21 + n22

        if t1 == 0 and t2 != 0:
            ma = float(n22) / float(t2)
        elif t1 != 0 and t2 == 0:
            ma = float(n11) / float(t1)
        elif t1 == 0 and t2 == 0:
            ma = 0
        else:
            ma = (float(n11) / float(t1) + float(n22) / float(t2)) / 2
        return ma * 100

    def IOU(self):
        intersection = np.logical_and(self.gt_image, self.pred_image)
        union = np.logical_or(self.gt_image, self.pred_image)
        iou_score = np.sum(intersection) / np.sum(union)
        return iou_score * 100

    def mean_IOU(self):
        n11, n12, n21, n22 = self.cm_terms()
        t1 = n11 + n12
        t2 = n21 + n22

        if (t1 + n21) == 0 and (t2 + n12) != 0:
            mIOU = float(n22) / float(t2 + n12)
        elif (t1 + n21) != 0 and (t2 + n12) == 0:
            mIOU = float(n11) / float(t1 + n21)
        else:
            mIOU = (float(n11) / float(t1 + n21) + float(n22) / float(t2 + n12)) / 2
        return mIOU * 100

    def fweight_IOU(self):
        n11, n12, n21, n22 = self.cm_terms()
        t1 = n11 + n12
        t2 = n21 + n22
        total_pixels = n11 + n12 + n21 + n22

        if total_pixels == 0:
            return 0

        fwIOU = 0
        if (n11 + n12 + n21) != 0:
            fwIOU += (t1 * n11) / float(n11 + n12 + n21)
        if (n12 + n21 + n22) != 0:
            fwIOU += (t2 * n22) / float(n12 + n21 + n22)

        fwIOU /= total_pixels

        return fwIOU * 100


In [14]:
# After training and testing your model, load and evaluate using the Evaluate class
if __name__ == "__main__":
    # Load test data and model

    model = tf.keras.models.load_model("model.h5", custom_objects={'iou': iou})

    # Initialize lists to store evaluation results
    pa_array = []
    ma_array = []
    IOU_array = []
    mIOU_array = []
    fwIOU_array = []

    # Evaluate each test image
    for i, (x_path, y_path) in tqdm(enumerate(zip(test_x, test_y)), total=len(test_x)):
        x = read_image(x_path)
        y = read_mask(y_path)
        y_pred = model.predict(np.expand_dims(x, axis=0))[0]
        y_pred = y_pred > 0.5
        y_pred = y_pred.astype(np.uint8)
        y = y.astype(np.uint8)

        # Convert to grayscale
        y_true_gray = y
        y_pred_gray = y_pred

        # Evaluate metrics
        evaluator = Evaluate(y_pred_gray, y_true_gray)
        pa = evaluator.pixel_accuracy()
        ma = evaluator.mean_accuracy()
        IOU = evaluator.IOU()
        mIOU = evaluator.mean_IOU()
        fwIOU = evaluator.fweight_IOU()

        # Append results to arrays
        pa_array.append(pa)
        ma_array.append(ma)
        IOU_array.append(IOU)
        mIOU_array.append(mIOU)
        fwIOU_array.append(fwIOU)

    # Calculate average metrics
    avg_pa = np.mean(pa_array)
    avg_ma = np.mean(ma_array)
    avg_IOU = np.mean(IOU_array)
    avg_mIOU = np.mean(mIOU_array)
    avg_fwIOU = np.mean(fwIOU_array)

    # Print and save results
    print("Average Pixel Accuracy (%): ", avg_pa)
    print("Average Mean Accuracy (%): ", avg_ma)
    print("Average Mean IoU (%): ", avg_mIOU)
    print("Average FWIoU (%): ", avg_fwIOU)

    # Write results to a file
    with open('evaluation_results.txt', 'w') as f:
        f.write(f"Average Pixel Accuracy (%): {avg_pa}\n")
        f.write(f"Average Mean Accuracy (%): {avg_ma}\n")
        f.write(f"Average Mean IoU (%): {avg_mIOU}\n")
        f.write(f"Average FWIoU (%): {avg_fwIOU}\n")


  0%|          | 0/53 [00:00<?, ?it/s]

1/1 [==============================] - 0s 265ms/step


  2%|▏         | 1/53 [00:01<01:05,  1.26s/it]

1/1 [==============================] - 0s 27ms/step


  4%|▍         | 2/53 [00:03<01:18,  1.55s/it]

...

1/1 [==============================] - 0s 18ms/step


 98%|█████████▊| 52/53 [01:01<00:01,  1.24s/it]

1/1 [==============================] - 0s 18ms/step


100%|██████████| 53/53 [01:02<00:00,  1.18s/it]

Average Pixel Accuracy (%):  99.84096311173349
Average Mean Accuracy (%):  94.26024616967217
Average Mean IoU (%):  90.3247489419954
Average FWIoU (%):  99.70054074812346
